<a href="https://colab.research.google.com/github/peruvianox/kT-Fp-MetCost/blob/main/KtFpActLen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Statistical Libraries

(SPM1D and Pingouin)

In [ ]:
pip install spm1d

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.5 MB 34.0 MB/s 


In [ ]:
pip install pingouin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 185 kB 17.1 MB/s 
     |████████████████████████████████| 9.8 MB 67.8 MB/s 
  Created wheel for pingouin: filename=pingouin-0.5.2-py3-none-any.whl size=196206 sha256=1379b22e8646d21fdf1b2ef02d5a1a1c3c7cb00f5cc5b8a547b59b111443bfd1
  Stored in directory: /root/.cache/pip/wheels/11/5a/63/a6d32fc26fa462c731f65480bfb98ff7bd39b8ebcb4bc6c2fe
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=dc50e0da663d080c5acb0d65c474c659b760b80037858a1b16ef3e2ea4eb5e8c
  Stored in directory: /root/.cache/pip/wheels/d6/64/cd/32819b511a488e4993f2fab909a95330289c3f4e0f6ef4676d
Successfully built pingouin littleutils
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2


## Load Data and Packages

In [ ]:
import os
import xml.etree.ElementTree as ET
from google.colab import drive
drive.mount("/content/drive")
DataFolder = '/content/drive/My Drive/ktFpMetCost/' # customize your own path!
os.chdir(DataFolder)

import matplotlib.pyplot as plt
import pandas as pd   
import fnmatch
import spm1d
from scipy.io import loadmat 
from scipy import interpolate
from scipy import signal
import numpy as np
from matplotlib.patches import Rectangle

Acts = pd.read_csv(DataFolder + 'Acts.csv').fillna(0.02) # fill empty (NaN) values with the minimum metabolic cost (0.02) 
FibLens = pd.read_csv(DataFolder + 'Fibs.csv')
plt.close('all')

Mounted at /content/drive


# Activation Analysis

In [ ]:
# simplify muscles with multiple lines of action
ActsSimp = Acts.reset_index(drop=True).drop(columns='Unnamed: 0')
UniMuscCols = np.unique(Acts['Muscles'])
SubjList = np.unique(Acts['Subj'])
conditions = ['Fm40', 'Fm20', 'NormF', 'Fp20', 'Fp40']
StrnLvl = [2, 3, 4, 6, 8]
Musc2Simp = ['add_mag', 'glut_max','glut_med','glut_min']

for S in SubjList:
  SubjInds = [x for x, z in enumerate(Acts['Subj'].tolist()) if z == S]
  for m in Musc2Simp:
    MuscInds = [x for x, z in enumerate(Acts['Muscles'].tolist()) if m in z]
    for C in conditions:
        CondInds = [x for x, z in enumerate(Acts['Cond'].tolist()) if z == C]
        for strn in StrnLvl:
            StrnInds = [x for x, z in enumerate(Acts['StrnLvl'].tolist()) if z == strn]

            # identify trials that match desired conditions
            match = list(set(MuscInds) & set(CondInds) & set(StrnInds) & set(SubjInds))
            match.sort()
            
            # sum values from invidual muscle lines of action
            Sum = np.sum(ActsSimp.iloc[match, 4:], axis=0)
            ActsSimp.iloc[match[0], 3] = m
            ActsSimp.iloc[match[0], 4:] = Sum
            ActsSimp.iloc[match[0]]
            
# drop the multi line-of-action muscles
Musc2Drop = []
for i, x in enumerate(ActsSimp['Muscles']):
  if '2' in x:
    Musc2Drop.append(i)
  elif '3' in x:
    Musc2Drop.append(i)

ActsSimp = ActsSimp.drop(index=Musc2Drop).reset_index(drop=True)

# save Activations
ActsSimp.to_csv(DataFolder + 'ActsSimp.csv')
Acts = ActsSimp

In [ ]:
# create "mean activation" variable for each condition
# mean activation == average activation across all muscles

# get all muscles for a given subjects, condition, & strain level
for S in SubjList:
  SubjInds = [x for x, z in enumerate(Acts['Subj'].tolist()) if z == S]
  for C in conditions:
      CondInds = [x for x, z in enumerate(Acts['Cond'].tolist()) if z == C]
      for strn in StrnLvl:
          StrnInds = [x for x, z in enumerate(Acts['StrnLvl'].tolist()) if z == strn]

          # identify trials that match desired conditions
          match = list(set(MuscInds) & set(CondInds) & set(StrnInds) & set(SubjInds))
          match.sort()
          
          # sum values from invidual muscle lines of action
          Mean = np.nanmean(ActsSimp.iloc[match, 4:], axis=0)
          Acts.loc[len(Acts)] = [S, strn, C, 'MEAN'] + list(Mean)
            

Cols = ['Subj','StrnLvl','Cond','Muscles']
ActAvg = pd.concat([Acts[Cols], pd.DataFrame(np.nanmean(Acts.iloc[:,4:], axis=1))], axis=1)
ActAvg.columns = Cols+['Act']

import seaborn as sb
ax = plt.figure(figsize=(10,10))
sb.boxplot(data=ActAvg[ActAvg['Muscles']=='MEAN'], 
           x='StrnLvl', y='Act', 
           hue='Cond')

## Plot Average Heat Maps

In [ ]:
# define function to plot average heat maps
plt.close('all')
fnt=24
import matplotlib.colors as mcolors
import pingouin as pg
conditions = ['Fm40', 'Fm20', 'NormF', 'Fp20', 'Fp40']
StrnLvls = ['0.02', '0.033','0.04','0.06','0.08']
plt.rcParams.update({'font.size': fnt})

def MakeHeatMap(musc):
  plt.figure(figsize=(12, 12))
  RelConds = ['-40', '-20', 'Norm', '+20', '+40']
  StrnLvl = [2, 3, 4, 6, 8]
  i = 0
  k = 1
  Map = np.zeros((5,5))
  Amap = np.empty((5,5), dtype=object)
  MuscInds = [x for x, z in enumerate(Acts['Muscles'].tolist()) if z == musc]
    
  for C in conditions:
      CondInds = [x for x, z in enumerate(Acts['Cond'].tolist()) if z == C]
      j = 0
      for strn in StrnLvl:
          # identify trials that match desired conditions
          StrnInds = [x for x, z in enumerate(Acts['StrnLvl'].tolist()) if z == strn]
          match = list(set(MuscInds) & set(CondInds) & set(StrnInds))
          match.sort()
          
          # create 2D array of average values
          Y = np.array(Acts.iloc[match, 4:])
          Amap[i, j] = Y.mean(axis=1) 
          Map[i, j] = Y.mean()
          
          j = j+1
          k = k+1
      
      i = i+1
      
      
  # define color normalization class
  class MidpointNormalize(mcolors.Normalize):
      def __init__(self, vmin=None, vmax=None, vcenter=None, clip=False):
          self.vcenter = vcenter
          super().__init__(vmin, vmax, clip)

      def __call__(self, value, clip=None):
          # Ignore masked values and all kinds of edge cases for a simple example...
          x, y = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1]
          return np.ma.masked_array(np.interp(value, x, y))

  # normalize top and bottom of heatmap color bar
  top = np.max(Map) - Map[2,1]
  bot = Map[2,1] - np.min(Map)
  M = max([top, bot])

  # set colorbar
  c = plt.pcolormesh(Map, cmap='Oranges')
  plt.colorbar(c, label='Activation')

  # set labels & ticks
  lbl = [0.5, 1.5, 2.5, 3.5, 4.5]
  plt.xticks(lbl, ['2%','3.3%','4%','6%','8%'], fontsize=fnt)
  plt.xlabel( r'$\epsilon_0$', fontsize=fnt+5)
  plt.yticks(lbl, RelConds, fontsize=fnt)
  plt.ylabel('$F_{P}$ Intensity', fontsize=fnt+5)
  plt.title(musc, fontsize=fnt+10)

  # get muscle indicies
  match = list(set(MuscInds))
  match.sort()
  A = pd.DataFrame(np.mean(Acts.iloc[match,4:], axis=1).reset_index()[0])
  AvgActs = pd.concat([Acts.iloc[match, 0:3].reset_index(), A], axis=1, 
                          ignore_index=True)
  
  # perform repeated measures anova on heatmaps
  D = AvgActs.iloc[:,1:5]
  D.columns = ['Subj','StrnLvl','Cond','Acts']
  HtMapRM = pg.rm_anova(data=D, dv='Acts', within=['Cond', 'StrnLvl'], subject='Subj', 
                        detailed=True, effsize='np2')
    
  plt.savefig('ActAvgHeatMaps/' + musc+'_Acts.pdf')
  plt.savefig('ActAvgHeatMaps/' + musc+'_Acts.png')
  return Map, Amap, HtMapRM

In [ ]:
# Make heatmaps for each muscle
UniMuscCols = np.unique(Acts['Muscles'])
UniMuscCols = np.append(UniMuscCols,'MEAN')
Map = {}
ArrMap = {}
RMAnova = {}
Vals = []
for muscle in UniMuscCols:
    [Map[muscle], ArrMap[muscle], RMAnova[muscle]] = MakeHeatMap(muscle)

In [ ]:
# HtMapRM['MEAN']

## Plot Gait Cycle Heat Maps for each Muscle

In [ ]:
# define function to plot gait cycle heat maps
def plotGCHeatMap(musc):
  # plot heatmap across the gait cycle, tendon stiffness, and condition

  fnt = 22 # set default fig font size
  plt.figure(figsize=(12, 12))
  plt.rcParams.update({'font.size': fnt})

  # define conditions and fig params
  Conds = ['Fm40','Fm20','NormF','Fp20','Fp40']
  CondNames = ['-40%','-20%','Norm','+20%','+40%']
  height = [0,5,10,15,20]
  fig, ax = plt.subplots()
  fig.set_figheight(15)
  fig.set_figwidth(15)

  # define color normalization class
  class MidpointNormalize(mcolors.Normalize):
    def __init__(self, vmin=None, vmax=None, vcenter=None, clip=False):
        self.vcenter = vcenter
        super().__init__(vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # ignore masked values and all kinds of edge cases for a simple example
        x, y = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

  # get average of all metabolic costs across the gait cycle for color normalization
  AllCosts = np.zeros((25, 100))
  for t in range(5):
      MuscInds = [x for x, z in enumerate(Acts['Muscles'].tolist()) if z == musc]
      cond = Conds[t]
      CondInds = [x for x, z in enumerate(Acts['Cond'].tolist()) if z == cond]
      match = list(set(MuscInds) & set(CondInds))
      match.sort()
      nrows = 1
      ncols = 100
      TSS = [2,3,4,6,8]
      for i in range(5): # plot individual heatmap rows
          y = np.arange(nrows+1)
          y = y + i + height[t]
          Y = np.array(Acts.iloc[match, 4:])
          B = np.array(Acts['StrnLvl'].iloc[match])
          AllCosts[y[0], :] = np.mean(Y[B==TSS[i]], axis=0).reshape(nrows, ncols)

  # loop through conditions and plot via colormesh
  for t in range(5):
      MuscInds = [x for x, z in enumerate(Acts['Muscles'].tolist()) if z == musc]
      cond = Conds[t]
      CondInds = [x for x, z in enumerate(Acts['Cond'].tolist()) if z == cond]
      match = list(set(MuscInds) & set(CondInds))
      match.sort()
      nrows = 1
      ncols = 100
      TSS = [2,3,4,6,8]

      for i in range(5): # plot individual heatmap rows
          x = np.arange(ncols)
          y = np.arange(nrows+1)
          y = y + i + height[t]
          
          Y = np.array(Acts.iloc[match, 4:])
          B = np.array(Acts['StrnLvl'].iloc[match])
          z = np.mean(Y[B==TSS[i]], axis=0).reshape(nrows, ncols)
          Z = np.repeat(z,2, axis=0)
          im = plt.pcolormesh(x, y, Z, shading='gouraud', cmap='Oranges', 
                               vmin=0, vmax=1)                    


  # run SPM stats
  conditionsR = ['Fp40', 'Fp20', 'NormF', 'Fm20', 'Fm40']
  alpha        = 0.05
  equal_var    = False
  Total = Acts.iloc[MuscInds, 4:]
  TotalMC = Acts[Acts['Muscles']=='MEAN'].reset_index()
  CondN = np.ones(len(TotalMC))
  Strns = np.array(TotalMC['StrnLvl'])
  Subjs = [int(float(x.replace('YA',''))) for x in TotalMC['Subj']]
  i = 0
  F = []
  Frm = []
  fi = []
  firm = []
  Ht = 25.25
  for x in conditionsR:
      CondInds = [x for x, z in enumerate(TotalMC['Cond'].tolist()) if z == conditionsR[i]]
      CondN[CondInds] = int(i + 1)
      i = i+1 
      
  F = spm1d.stats.anova2(Total, CondN, Strns, equal_var=True)  #between-subjects
  Frm = spm1d.stats.anova2rm(Total, CondN, Strns, Subjs)  #within-subjects (repeated-measures)
  fi.append(F.inference(alpha))
  firm.append(Frm.inference(alpha))
  Fi = F.inference(alpha)
  Firm = Frm.inference(alpha) 

  for x in range(len(Firm[0].z)):
      # effect of interaction?
      if Firm[2].z[x] > Firm[2].zstar:
          ax.add_patch(Rectangle((x, 25), 1, 2, 
                          facecolor='k', alpha=0.4))

      # effect of strain
      if Firm[1].z[x] > Firm[1].zstar:
          ax.add_patch(Rectangle((x, Ht), 1, 0.5, 
                          facecolor='k'))
          
      #effect of condition
      if Firm[0].z[x] > Firm[0].zstar:
          ax.add_patch(Rectangle((x, Ht+1), 1, 0.5, 
                      facecolor='k'))
          


  ax.text(-4, Ht+0.25, r'$\epsilon_0$', va='center', fontsize=fnt)
  ax.text(-4, Ht+1.25, r'$F_P$', va='center', fontsize=fnt)

  # edit axes
  plt.colorbar(im, label='Instantaneous Activation Level', orientation='horizontal', pad=0.08, 
               boundaries=np.linspace(0, 1, 100), 
               ticks=np.round(np.linspace(0, 1, 5, endpoint=True), 3))

  ax.set_yticks(np.linspace(0.5, 24.5, 25))
  TSSvals = ['2%','3.3%','4%','6%','8%']
  ax.set_yticklabels(np.tile(TSSvals, 5), fontsize=fnt-5)
  ax.set_ylabel(r'$\epsilon_0$', fontsize=fnt+8)
  ax.set_xlabel('% Gait Cycle', fontsize=fnt+3)
  ax.set_xlim(0,99)
  ax.set_ylim(0,27)
  ax.hlines([5, 10, 15, 20], 0, 100, 'w', linewidth=5)
  ax.hlines(25, -5, 100, 'k', linewidth=2)
  ax.set_title(musc, fontsize=fnt+20, pad=25, weight = 'bold')
  yvals = [2.5, 7.5, 12.5, 17.5, 22.5]
  for i in range(5):
      ax.text(101, yvals[i], CondNames[i], rotation='vertical', ha='center', va='center', fontsize=fnt)
  ax.text(105, 13, '$F_{P}$ Intensity', rotation='vertical', ha='center', va='center', fontsize=fnt+5)

  fig.savefig('ActGCHeatMaps/' + musc+'_ActHeatmap.png')
  fig.savefig('ActGCHeatMaps/' + musc+'_ActHeatmap.pdf')

In [ ]:
# generate figures for all muscles
Go = 1
if Go == 1:
  AllMuscles = np.unique(Acts['Muscles'])
  for M in AllMuscles: 
      print(M)
      plotGCHeatMap(M)

# Make Table of Activation levels

In [ ]:
# make table of ranked metabolic costs
muscles = np.unique(Acts['Muscles'])
MuscleData = {}
Strn = 3
StrnInds = [x for x, z in enumerate(Acts['StrnLvl'].tolist()) if z == Strn]
cond = 'NormF'
CondInds = [x for x, z in enumerate(Acts['Cond'].tolist()) if z == cond]
match = list(set(CondInds) & set(StrnInds))
Defaults = Acts.iloc[match, :]
muscles = np.unique(Acts['Muscles'])

M = {}
M_Avg = {}
M_Tot = {}
TotCost = []
M_Conds = {}
M_Strns = {}
for m in muscles: 
    # muscle average data from default condition and strain level
    MuscInds = [x for x, z in enumerate(Defaults['Muscles'].tolist()) if z == m]
    match = list(set(MuscInds))
    M[m] = Defaults.iloc[match, 5:]
    M_Avg[m] = M[m].mean()
    M_Tot[m] = M_Avg[m].mean()
    TotCost.append(M_Avg[m].mean())

    MuscInds = [x for x, z in enumerate(Acts['Muscles'].tolist()) if z == m]
    
    # save condition averages
    Conds = ['Fm40','Fm20','NormF','Fp20','Fp40']
    CondVals = []
    for C in Conds: 
        CondInds = [x for x, z in enumerate(Acts['Cond'].tolist()) if z == C]
        Strn = 3
        StrnInds = [x for x, z in enumerate(Acts['StrnLvl'].tolist()) if z == Strn]
        match = list(set(CondInds) & set(StrnInds) & set(MuscInds))
        condVals = np.mean(np.array(Acts.iloc[match, 5:]))#.mean().mean()
        CondVals.append(condVals)
        del condVals, match, StrnInds, CondInds

    M_Conds[m] = CondVals

    # save strain averages
    Strns = [2, 3, 4, 6, 8]
    StrnVals = []
    for S in Strns: 
        StrnInds = [x for x, z in enumerate(Acts['StrnLvl'].tolist()) if z == S]
        Cond = 'NormF'
        CondInds = [x for x, z in enumerate(Acts['Cond'].tolist()) if z == Cond]
        match = list(set(CondInds) & set(StrnInds) & set(MuscInds))
        strnVals = Acts.iloc[match, 5:].mean().mean()
        StrnVals.append(strnVals)
        del strnVals, match, StrnInds, CondInds

    M_Strns[m] = StrnVals

In [ ]:
from scipy.signal import correlate2d
T = pd.DataFrame(columns = ['Muscle','DefAct','pFp','nFp', 'pKt', 'nKt', 'pInt', 'nInt'])
denom = M_Tot['MEAN'] # set denominator for relative metabolic cost
TotMap = Map['MEAN'] # set total map for 2D correlation -> goodness of fit
MaxCorr = correlate2d(TotMap, TotMap, mode='valid')
Cond = 0
StrnLvl = 1
Interaction = 2
for i, m in enumerate(muscles): 
  T.loc[i] = [m, M_Tot[m], RMAnova[m]['p-unc'][Cond], RMAnova[m]['np2'][Cond],
              RMAnova[m]['p-unc'][StrnLvl], RMAnova[m]['np2'][StrnLvl], 
              RMAnova[m]['p-unc'][Interaction], RMAnova[m]['np2'][Interaction]]

T.sort_values(by=['DefAct'], ascending=False).reset_index(drop=True).to_csv('RankAct.csv')

In [ ]:
T

# Fiber Length Analysis

In [ ]:
# load fiber lengths
FibLens = pd.read_csv(DataFolder + 'Fibs.csv')
# simplify muscles with multiple lines of action
FibLensSimp = FibLens.reset_index(drop=True).drop(columns='Unnamed: 0')
UniMuscCols = np.unique(FibLens['Muscles'])
SubjList = np.unique(FibLens['Subj'])
conditions = ['Fm40', 'Fm20', 'NormF', 'Fp20', 'Fp40']
StrnLvl = [2, 3, 4, 6, 8]
Musc2Simp = ['add_mag', 'glut_max','glut_med','glut_min']

# processing loop
for S in SubjList:
  SubjInds = [x for x, z in enumerate(FibLens['Subj'].tolist()) if z == S]

  for m in Musc2Simp:
    MuscInds = [x for x, z in enumerate(FibLens['Muscles'].tolist()) if m in z]
    
    for C in conditions:
        CondInds = [x for x, z in enumerate(FibLens['Cond'].tolist()) if z == C]

        for strn in StrnLvl:
            StrnInds = [x for x, z in enumerate(FibLens['StrnLvl'].tolist()) if z == strn]

            # identify trials that match desired conditions
            match = list(set(MuscInds) & set(CondInds) & set(StrnInds) & set(SubjInds))
            match.sort()
            
            # sum values from invidual muscle lines of action
            Avg = np.mean(FibLensSimp.iloc[match, 4:], axis=0)
            FibLensSimp.iloc[match[0], 3] = m
            FibLensSimp.iloc[match[0], 4:] = Avg
            FibLensSimp.iloc[match[0]]
            
# drop the multi line-of-action muscles
Musc2Drop = []
for i, x in enumerate(FibLensSimp['Muscles']):
  if '2' in x:
    Musc2Drop.append(i)
  elif '3' in x:
    Musc2Drop.append(i)

FibLensSimp = FibLensSimp.drop(index=Musc2Drop).reset_index(drop=True)

# save Activations
FibLensSimp.to_csv(DataFolder + 'FibLensSimp.csv')
FibLens = FibLensSimp

## Average Heat Maps

In [ ]:
# define function to plot average heat maps
plt.close('all')
fnt=24
import matplotlib.colors as mcolors
import pingouin as pg
conditions = ['Fm40', 'Fm20', 'NormF', 'Fp20', 'Fp40']
StrnLvls = ['0.02', '0.033','0.04','0.06','0.08']
plt.rcParams.update({'font.size': fnt})

def MakeHeatMap(musc):
  plt.figure(figsize=(12, 12))
  RelConds = ['-40', '-20', 'Norm', '+20', '+40']
  StrnLvl = [2, 3, 4, 6, 8]
  i = 0
  k = 1
  Map = np.zeros((5,5))
  Amap = np.empty((5,5), dtype=object)

  MuscInds = [x for x, z in enumerate(FibLens['Muscles'].tolist()) if z == musc]
    
  for C in conditions:
      CondInds = [x for x, z in enumerate(FibLens['Cond'].tolist()) if z == C]
      j = 0
      for strn in StrnLvl:
          # identify trials that match desired conditions
          StrnInds = [x for x, z in enumerate(FibLens['StrnLvl'].tolist()) if z == strn]
          match = list(set(MuscInds) & set(CondInds) & set(StrnInds))
          match.sort()
          
          # create 2D array of average values
          Y = np.array(FibLens.iloc[match, 4:])
          Amap[i, j] = Y.mean(axis=1) 
          Map[i, j] = Y.mean()
          
          j = j+1
          k = k+1
      
      i = i+1
      
      
  # define color normalization class
  class MidpointNormalize(mcolors.Normalize):
      def __init__(self, vmin=None, vmax=None, vcenter=None, clip=False):
          self.vcenter = vcenter
          super().__init__(vmin, vmax, clip)

      def __call__(self, value, clip=None):
          # Ignore masked values and all kinds of edge cases for a simple example...
          x, y = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1]
          return np.ma.masked_array(np.interp(value, x, y))

  # normalize top and bottom of heatmap color bar
  top = np.max(Map) - Map[2,1]
  bot = Map[2,1] - np.min(Map)
  M = max([top, bot])
  midnorm = MidpointNormalize(vmin = Map[2,1] - M, vcenter=Map[2,1], vmax=Map[2,1] + M)
  # apply colorbar
  c = plt.pcolormesh(Map, cmap='PRGn', norm=midnorm)
  plt.colorbar(c, label='Relative Length')

  # setup labels & ticks
  lbl = [0.5, 1.5, 2.5, 3.5, 4.5]
  plt.xticks(lbl, ['2%','3.3%','4%','6%','8%'], fontsize=fnt)
  plt.xlabel( r'$\epsilon_0$', fontsize=fnt+5)
  plt.yticks(lbl, RelConds, fontsize=fnt)
  plt.ylabel('$F_{P}$ Intensity', fontsize=fnt+5)
  plt.title(musc, fontsize=fnt+10)

  # get muscles indicies from original dataset
  match = list(set(MuscInds))
  match.sort()
  A = pd.DataFrame(np.mean(FibLens.iloc[match,4:], axis=1).reset_index()[0])
  AvgFibLens = pd.concat([FibLens.iloc[match, 0:3].reset_index(), A], axis=1, 
                          ignore_index=True)
  
  # perform repeated measures anova on heatmaps
  D = AvgFibLens.iloc[:,1:5]
  D.columns = ['Subj','StrnLvl','Cond','FibLens']
  HtMapRM = pg.rm_anova(data=D, dv='FibLens', within=['Cond', 'StrnLvl'], subject='Subj', 
                        detailed=True, effsize='np2')
    
  plt.savefig('ActAvgHeatMaps/' + musc+'_FibLens.pdf')
  plt.savefig('ActAvgHeatMaps/' + musc+'_FibLens.png')
  return Map, Amap, HtMapRM

In [ ]:
# Make heatmaps for each muscle
UniMuscCols = np.unique(FibLens['Muscles'])
Map = {}
ArrMap = {}
RMAnova = {}
Vals = []
for muscle in UniMuscCols:
    [Map[muscle], ArrMap[muscle], RMAnova[muscle]] = MakeHeatMap(muscle)

## Gait Cycle Heat Maps

In [ ]:
# define function to plot gait cycle heat maps
def plotGCHeatMap(musc):
  # plot heatmap across the gait cycle, tendon stiffness, and condition

  fnt = 22 # set default fig font size
  plt.rcParams.update({'font.size': fnt})

  # define conditions and fig params
  Conds = ['Fm40','Fm20','NormF','Fp20','Fp40']
  CondNames = ['-40%','-20%','Norm','+20%','+40%']
  height = [0,5,10,15,20]
  fig, ax = plt.subplots()
  fig.set_figheight(15)
  fig.set_figwidth(15)

  # define color normalization class
  class MidpointNormalize(mcolors.Normalize):
    def __init__(self, vmin=None, vmax=None, vcenter=None, clip=False):
        self.vcenter = vcenter
        super().__init__(vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # ignore masked values and all kinds of edge cases for a simple example
        x, y = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

  # get average of all metabolic costs across the gait cycle for color normalization
  AllCosts = np.zeros((25, 100))
  for t in range(5):
      MuscInds = [x for x, z in enumerate(FibLens['Muscles'].tolist()) if z == musc]
      cond = Conds[t]
      CondInds = [x for x, z in enumerate(FibLens['Cond'].tolist()) if z == cond]
      match = list(set(MuscInds) & set(CondInds))
      match.sort()
      nrows = 1
      ncols = 100
      TSS = [2,3,4,6,8]
      for i in range(5): # plot individual heatmap rows
          y = np.arange(nrows+1)
          y = y + i + height[t]
          Y = np.array(FibLens.iloc[match, 4:])
          B = np.array(FibLens['StrnLvl'].iloc[match])
          AllCosts[y[0], :] = np.mean(Y[B==TSS[i]], axis=0).reshape(nrows, ncols)

  top = np.max(AllCosts) - np.mean(AllCosts)
  bot = np.mean(AllCosts) - np.min(AllCosts)
  M = max([top, bot]) # calculate over/under margin (M) to use for colorbar normalization
  midnorm = MidpointNormalize(vmin=np.mean(AllCosts) - M, vcenter=np.mean(AllCosts), vmax=np.mean(AllCosts) + M)

  # loop through conditions and plot via colormesh
  for t in range(5):
      MuscInds = [x for x, z in enumerate(FibLens['Muscles'].tolist()) if z == musc]
      cond = Conds[t]
      CondInds = [x for x, z in enumerate(FibLens['Cond'].tolist()) if z == cond]
      match = list(set(MuscInds) & set(CondInds))
      match.sort()
      nrows = 1
      ncols = 100
      TSS = [2,3,4,6,8]

      for i in range(5): # plot individual heatmap rows
          x = np.arange(ncols)
          y = np.arange(nrows+1)
          y = y + i + height[t]
          
          Y = np.array(FibLens.iloc[match, 4:])
          B = np.array(FibLens['StrnLvl'].iloc[match])
          z = np.mean(Y[B==TSS[i]], axis=0).reshape(nrows, ncols)
          Z = np.repeat(z,2, axis=0)
          im = plt.pcolormesh(x, y, Z, shading='gouraud', cmap='PRGn', norm=midnorm, 
                              vmin=np.mean(AllCosts) - M, vmax=np.mean(AllCosts) + M)


  # run SPM stats
  conditionsR = ['Fp40', 'Fp20', 'NormF', 'Fm20', 'Fm40']
  alpha        = 0.05
  equal_var    = False
  Total = FibLens.iloc[MuscInds, 4:]
  TotalInfo = FibLens[FibLens['Muscles']==musc].reset_index()
  CondN = np.ones(len(TotalInfo))
  Strns = np.array(TotalInfo['StrnLvl'])
  Subjs = [int(float(x.replace('YA',''))) for x in TotalInfo['Subj']]
  i = 0
  F = []
  Frm = []
  fi = []
  firm = []
  Ht = 25.25
  for x in conditionsR:
      CondInds = [x for x, z in enumerate(TotalInfo['Cond'].tolist()) if z == conditionsR[i]]
      CondN[CondInds] = int(i + 1)
      i = i+1 
      
  F = spm1d.stats.anova2(Total, CondN, Strns, equal_var=True)  #between-subjects
  Frm = spm1d.stats.anova2rm(Total, CondN, Strns, Subjs)  #within-subjects (repeated-measures)
  fi.append(F.inference(alpha))
  firm.append(Frm.inference(alpha))
  Fi = F.inference(alpha)
  Firm = Frm.inference(alpha) 

  for x in range(len(Firm[0].z)):
      # effect of interaction?
      if Firm[2].z[x] > Firm[2].zstar:
          ax.add_patch(Rectangle((x, 25), 1, 2, 
                          facecolor='k', alpha = 0.4))
          
      # effect of strain
      if Firm[1].z[x] > Firm[1].zstar:
          ax.add_patch(Rectangle((x, Ht), 1, 0.5, 
                          facecolor='k'))
          
      #effect of condition
      if Firm[0].z[x] > Firm[0].zstar:
          ax.add_patch(Rectangle((x, Ht+1), 1, 0.5, 
                      facecolor='k'))
          


  ax.text(-4, Ht+0.25, r'$\epsilon_0$', va='center', fontsize=fnt)
  ax.text(-4, Ht+1.25, r'$F_P$', va='center', fontsize=fnt)

  # edit axes
  cb = plt.colorbar(im, label='Relative Fiber Length ' + r'$(L/L_{0})$', orientation='horizontal', pad=0.08) 
  ax.set_yticks(np.linspace(0.5, 24.5, 25))
  TSSvals = ['2%','3.3%','4%','6%','8%']
  ax.set_yticklabels(np.tile(TSSvals, 5), fontsize=fnt-5)
  ax.set_ylabel(r'$\epsilon_0$', fontsize=fnt+8)
  ax.set_xlabel('% Gait Cycle', fontsize=fnt+3)
  ax.set_xlim(0,99)
  ax.set_ylim(0,27)
  ax.hlines([5, 10, 15, 20], 0, 100, 'w', linewidth=5)
  ax.hlines(25, -5, 100, 'k', linewidth=2)
  ax.set_title(musc, fontsize=fnt+20, pad=25, weight = 'bold')
  yvals = [2.5, 7.5, 12.5, 17.5, 22.5]
  for i in range(5):
      ax.text(101, yvals[i], CondNames[i], rotation='vertical', ha='center', va='center', fontsize=fnt)
  ax.text(105, 13, '$F_{P}$ Intensity', rotation='vertical', ha='center', va='center', fontsize=fnt+5)

  fig.savefig('ActGCHeatMaps/' + musc+'_GCHeatmap.png')
  fig.savefig('ActGCHeatMaps/' + musc+'_GCHeatmap.pdf')

In [ ]:
# generate figures for all muscles
# plotGCHeatMap('Total')
Go = 1
if Go == 1:
  AllMuscles = np.unique(FibLens['Muscles'])
  for M in AllMuscles: 
      print(M)
      plotGCHeatMap(M)

# Make Table of Fiber Lengths

In [ ]:
# make table of ranked metabolic costs
muscles = np.unique(FibLens['Muscles'])
MuscleData = {}
Strn = 3
StrnInds = [x for x, z in enumerate(FibLens['StrnLvl'].tolist()) if z == Strn]
cond = 'NormF'
CondInds = [x for x, z in enumerate(FibLens['Cond'].tolist()) if z == cond]
match = list(set(CondInds) & set(StrnInds))
Defaults = FibLens.iloc[match, :]
muscles = np.unique(FibLens['Muscles'])

M = {}
M_Avg = {}
M_Tot = {}
TotCost = []
M_Conds = {}
M_Strns = {}
for m in muscles: 
    # muscle average data from default condition and strain level
    MuscInds = [x for x, z in enumerate(Defaults['Muscles'].tolist()) if z == m]
    match = list(set(MuscInds))
    M[m] = Defaults.iloc[match, 5:]
    M_Avg[m] = M[m].mean()
    M_Tot[m] = M_Avg[m].mean()
    TotCost.append(M_Avg[m].mean())

    MuscInds = [x for x, z in enumerate(Acts['Muscles'].tolist()) if z == m]
    
    # save condition averages
    Conds = ['Fm40','Fm20','NormF','Fp20','Fp40']
    CondVals = []
    for C in Conds: 
        CondInds = [x for x, z in enumerate(Acts['Cond'].tolist()) if z == C]
        Strn = 3
        StrnInds = [x for x, z in enumerate(Acts['StrnLvl'].tolist()) if z == Strn]
        match = list(set(CondInds) & set(StrnInds) & set(MuscInds))
        condVals = np.mean(np.array(Acts.iloc[match, 5:]))#.mean().mean()
        CondVals.append(condVals)
        del condVals, match, StrnInds, CondInds

    M_Conds[m] = CondVals

    # save strain averages
    Strns = [2, 3, 4, 6, 8]
    StrnVals = []
    for S in Strns: 
        StrnInds = [x for x, z in enumerate(Acts['StrnLvl'].tolist()) if z == S]
        Cond = 'NormF'
        CondInds = [x for x, z in enumerate(Acts['Cond'].tolist()) if z == Cond]
        match = list(set(CondInds) & set(StrnInds) & set(MuscInds))
        strnVals = Acts.iloc[match, 5:].mean().mean()
        StrnVals.append(strnVals)
        del strnVals, match, StrnInds, CondInds

    M_Strns[m] = StrnVals

In [ ]:
# from scipy.signal import correlate2d
T = pd.DataFrame(columns = ['Muscle','DefLen','pFp','nFp', 'pKt', 'nKt', 'pInt', 'nInt'])
Cond = 0
StrnLvl = 1
Interaction = 2
for i, m in enumerate(muscles): 
  T.loc[i] = [m, M_Tot[m], RMAnova[m]['p-unc'][Cond], RMAnova[m]['np2'][Cond],
              RMAnova[m]['p-unc'][StrnLvl], RMAnova[m]['np2'][StrnLvl], 
              RMAnova[m]['p-unc'][Interaction], RMAnova[m]['np2'][Interaction]]

T.sort_values(by=['DefLen'], ascending=False).reset_index(drop=True).to_csv('RankFibLen.csv')